# 第8章 ディープラーニングを用いた因果探索

#### ・本章ではディープラーニングを用いた因果探索について解説、実装する .

#### 2020年において、因果推論の分野でもディープラーニングを利用した研究が進んでいる .　具体例として、

#### グラフニューラルネットワークを用いた因果探索
#### 深層強化学習を用いた因果探索
#### GAN(Generative Adversarial Networks)を用いた因果探索

#### などが挙げられる .

#### ・本章ではディープラーニングを用いた因果探索手法の中でも、"SAM(Structural Agnostic Model)"について解説、実装する .　SAMはGANを用いた因果探索手法である .

#### ・本章の内容はPyTorchを用いたディープラーニングとGANの実装経験がないと理解が難しい部分が多いらしいです .

#### ・本章で出てくる実装はSAM論文の著者らのGitHubのコードを一部参考、使用している .

![alt text](pict1.png)

## GAN(Generative Adversarial Networks)とは

#### ・GANは大量の画像データを学習に使用し、実際には存在していない架空の画像を生成する技術として有名である .

#### GANは生成器(Generator,以下G)と呼ばれるニューラルネットワークと、識別器(Discriminator,以下D)と呼ばれる2種類のニューラルネットワークから構成される .

#### ・画像を生成する際に使用するのは、「学習済みの生成器G」のみである .　学習済みの生成器Gに入力としてノイズを与えると、